# Model Predictive Control (MPC) for Constrained Optimal Control

This notebook demonstrates **Model Predictive Control (MPC)** for systems with input and state constraints.

**When to Use MPC**:
- Input constraints (e.g., actuator saturation)
- State constraints (e.g., safety limits, obstacle avoidance)
- Time-varying systems or references
- Optimal trajectory generation

**Key Advantage over LQR**: MPC explicitly handles constraints, LQR does not.

**Reference**: Mayne et al. (2000), "Constrained model predictive control: Stability and optimality"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pykal import DynamicalSystem
from pykal.algorithm_library.controllers.mpc import MPC
from pykal.algorithm_library.controllers.lqr import LQR

np.random.seed(42)

## 1. Problem Setup: Double Integrator with Constraints

### System Dynamics
$$\begin{bmatrix} x_{k+1} \\ v_{k+1} \end{bmatrix} = \begin{bmatrix} 1 & \Delta t \\ 0 & 1 \end{bmatrix} \begin{bmatrix} x_k \\ v_k \end{bmatrix} + \begin{bmatrix} \frac{\Delta t^2}{2} \\ \Delta t \end{bmatrix} u_k$$

where:
- $x$ = position
- $v$ = velocity  
- $u$ = acceleration (control input)

### Constraints
- **Input**: $|u| \leq 1.0$ (actuator saturation)
- **State**: $|v| \leq 2.0$ (velocity limit)

In [ ]:
# Discrete-time double integrator
dt = 0.1  # Timestep
A = np.array([[1.0, dt],
              [0.0, 1.0]])
B = np.array([[0.5 * dt**2],
              [dt]])

# Cost matrices
Q = np.diag([10.0, 1.0])  # State cost (position >> velocity)
R = np.array([[1.0]])      # Control cost

# MPC parameters
N = 20  # Prediction horizon

# Constraints
u_min = np.array([-1.0])
u_max = np.array([1.0])
x_min = np.array([-np.inf, -2.0])  # Only velocity constrained
x_max = np.array([np.inf, 2.0])

print("System Dynamics:")
print(f"  dt = {dt}s")
print(f"  A =\n{A}")
print(f"  B =\n{B}")
print(f"\nMPC Parameters:")
print(f"  Horizon N = {N}")
print(f"  Input constraints: {u_min[0]} ≤ u ≤ {u_max[0]}")
print(f"  Velocity constraints: {x_min[1]} ≤ v ≤ {x_max[1]}")

## 2. Compute LQR Gain for Comparison

LQR provides optimal unconstrained control.

In [ ]:
# Compute LQR gain
K_lqr, P_lqr = LQR.compute_gain(A, B, Q, R)

print("LQR Gain:")
print(f"  K = {K_lqr}")
print(f"\nNote: LQR does NOT respect constraints!")

## 3. Simulation Setup

In [ ]:
# Simulation parameters
T_sim = 10.0  # Simulation duration (s)
t_sim = np.arange(0, T_sim, dt)
N_steps = len(t_sim)

# Initial condition (far from origin)
x0 = np.array([5.0, 0.0])  # Start at x=5m, stationary

# Reference trajectory (step to origin)
x_ref = np.zeros((N_steps, 2))

print(f"Simulation:")
print(f"  Duration: {T_sim}s")
print(f"  Steps: {N_steps}")
print(f"  Initial state: {x0}")
print(f"  Reference: origin [0, 0]")

## 4. Run MPC Controller

In [ ]:
# MPC simulation
x_mpc = np.zeros((N_steps, 2))
u_mpc = np.zeros(N_steps)
x_mpc[0] = x0

print("Running MPC simulation...")

for k in range(N_steps - 1):
    # Current state
    xk = x_mpc[k]
    
    # Compute MPC control
    u_opt, x_pred = MPC.solve_single_step(
        xk=xk,
        xref=x_ref[k],
        A=A,
        B=B,
        Q=Q,
        R=R,
        P=P_lqr,  # Terminal cost (from LQR DARE)
        N=N,
        u_min=u_min,
        u_max=u_max,
        x_min=x_min,
        x_max=x_max
    )
    
    # Apply first control from sequence
    u_mpc[k] = u_opt
    
    # Propagate dynamics
    x_mpc[k+1] = A @ xk + B @ np.array([u_opt])

u_mpc[-1] = u_mpc[-2]  # Hold last control

print("MPC simulation complete!")
print(f"\nFinal state: {x_mpc[-1]}")
print(f"Final error: {np.linalg.norm(x_mpc[-1] - x_ref[-1]):.4f}")

## 5. Run LQR Controller (For Comparison)

In [ ]:
# LQR simulation
x_lqr = np.zeros((N_steps, 2))
u_lqr = np.zeros(N_steps)
x_lqr[0] = x0

print("Running LQR simulation...")

for k in range(N_steps - 1):
    # Current state
    xk = x_lqr[k]
    
    # LQR control (state feedback)
    u_lqr[k] = (-K_lqr @ (xk - x_ref[k]))[0]
    
    # Propagate dynamics (NO SATURATION - LQR ignores constraints!)
    x_lqr[k+1] = A @ xk + B @ np.array([u_lqr[k]])

u_lqr[-1] = u_lqr[-2]

print("LQR simulation complete!")
print(f"\nFinal state: {x_lqr[-1]}")
print(f"Final error: {np.linalg.norm(x_lqr[-1] - x_ref[-1]):.4f}")

# Check constraint violations
u_violations_lqr = np.sum((u_lqr < u_min[0]) | (u_lqr > u_max[0]))
v_violations_lqr = np.sum((x_lqr[:, 1] < x_min[1]) | (x_lqr[:, 1] > x_max[1]))

print(f"\nLQR Constraint Violations:")
print(f"  Input violations: {u_violations_lqr}/{N_steps}")
print(f"  Velocity violations: {v_violations_lqr}/{N_steps}")

## 6. Visualize Results

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(14, 10))

# Position trajectory
axes[0, 0].plot(t_sim, x_mpc[:, 0], 'b-', linewidth=2, label='MPC')
axes[0, 0].plot(t_sim, x_lqr[:, 0], 'r--', linewidth=2, label='LQR')
axes[0, 0].axhline(y=0, color='k', linestyle=':', label='Reference')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Position (m)')
axes[0, 0].set_title('Position Tracking', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Velocity trajectory
axes[0, 1].plot(t_sim, x_mpc[:, 1], 'b-', linewidth=2, label='MPC')
axes[0, 1].plot(t_sim, x_lqr[:, 1], 'r--', linewidth=2, label='LQR')
axes[0, 1].axhline(y=0, color='k', linestyle=':', label='Reference')
axes[0, 1].axhline(y=x_max[1], color='orange', linestyle='--', label='Velocity limit')
axes[0, 1].axhline(y=x_min[1], color='orange', linestyle='--')
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Velocity (m/s)')
axes[0, 1].set_title('Velocity (with constraints)', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Control input
axes[1, 0].plot(t_sim, u_mpc, 'b-', linewidth=2, label='MPC')
axes[1, 0].plot(t_sim, u_lqr, 'r--', linewidth=2, label='LQR')
axes[1, 0].axhline(y=u_max[0], color='red', linestyle='--', linewidth=2, label='Input limit')
axes[1, 0].axhline(y=u_min[0], color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Control Input')
axes[1, 0].set_title('Control Input (with constraints)', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Phase portrait
axes[1, 1].plot(x_mpc[:, 0], x_mpc[:, 1], 'b-', linewidth=2, label='MPC')
axes[1, 1].plot(x_lqr[:, 0], x_lqr[:, 1], 'r--', linewidth=2, label='LQR')
axes[1, 1].plot(x0[0], x0[1], 'go', markersize=10, label='Start')
axes[1, 1].plot(0, 0, 'r*', markersize=15, label='Goal')
axes[1, 1].axhline(y=x_max[1], color='orange', linestyle='--', label='Velocity limit')
axes[1, 1].axhline(y=x_min[1], color='orange', linestyle='--')
axes[1, 1].set_xlabel('Position (m)')
axes[1, 1].set_ylabel('Velocity (m/s)')
axes[1, 1].set_title('Phase Portrait', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True)

# Position error
error_mpc = np.abs(x_mpc[:, 0] - x_ref[:, 0])
error_lqr = np.abs(x_lqr[:, 0] - x_ref[:, 0])
axes[2, 0].plot(t_sim, error_mpc, 'b-', linewidth=2, label='MPC')
axes[2, 0].plot(t_sim, error_lqr, 'r--', linewidth=2, label='LQR')
axes[2, 0].set_xlabel('Time (s)')
axes[2, 0].set_ylabel('Position Error (m)')
axes[2, 0].set_title('Tracking Error', fontweight='bold')
axes[2, 0].set_yscale('log')
axes[2, 0].legend()
axes[2, 0].grid(True, which='both')

# Cumulative cost
cost_mpc = np.cumsum(x_mpc[:, 0]**2 * Q[0, 0] + x_mpc[:, 1]**2 * Q[1, 1] + u_mpc**2 * R[0, 0])
cost_lqr = np.cumsum(x_lqr[:, 0]**2 * Q[0, 0] + x_lqr[:, 1]**2 * Q[1, 1] + u_lqr**2 * R[0, 0])
axes[2, 1].plot(t_sim, cost_mpc, 'b-', linewidth=2, label='MPC')
axes[2, 1].plot(t_sim, cost_lqr, 'r--', linewidth=2, label='LQR')
axes[2, 1].set_xlabel('Time (s)')
axes[2, 1].set_ylabel('Cumulative Cost')
axes[2, 1].set_title('Cost Evolution', fontweight='bold')
axes[2, 1].legend()
axes[2, 1].grid(True)

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("- MPC respects ALL constraints (input and velocity)")
print("- LQR violates constraints (visible in red dashed lines)")
print("- MPC achieves similar or better tracking despite constraints")

## 7. Constraint Violation Analysis

In [ ]:
# MPC constraint violations (should be zero)
u_violations_mpc = np.sum((u_mpc < u_min[0] - 1e-6) | (u_mpc > u_max[0] + 1e-6))
v_violations_mpc = np.sum((x_mpc[:, 1] < x_min[1] - 1e-6) | (x_mpc[:, 1] > x_max[1] + 1e-6))

print("=== Constraint Satisfaction Analysis ===")
print(f"\nMPC:")
print(f"  Input violations: {u_violations_mpc}/{N_steps} (should be 0)")
print(f"  Velocity violations: {v_violations_mpc}/{N_steps} (should be 0)")
print(f"  Max input: {u_mpc.max():.3f} (limit: {u_max[0]})")
print(f"  Min input: {u_mpc.min():.3f} (limit: {u_min[0]})")
print(f"  Max velocity: {x_mpc[:, 1].max():.3f} (limit: {x_max[1]})")
print(f"  Min velocity: {x_mpc[:, 1].min():.3f} (limit: {x_min[1]})")

print(f"\nLQR:")
print(f"  Input violations: {u_violations_lqr}/{N_steps}")
print(f"  Velocity violations: {v_violations_lqr}/{N_steps}")
print(f"  Max input: {u_lqr.max():.3f} (exceeds limit!)")
print(f"  Min input: {u_lqr.min():.3f} (exceeds limit!)")
print(f"  Max velocity: {x_lqr[:, 1].max():.3f}")
print(f"  Min velocity: {x_lqr[:, 1].min():.3f}")

## 8. Using MPC with DynamicalSystem

In [ ]:
# Create MPC as a DynamicalSystem
mpc_system = DynamicalSystem(
    f=MPC.simple_f,
    h=MPC.simple_h,
    state_name='mpc_output'
)

# Parameters
param_dict = {
    'xk': x0,
    'xref': np.zeros(2),
    'A': A,
    'B': B,
    'Q': Q,
    'R': R,
    'P': P_lqr,
    'N': N,
    'u_min': u_min,
    'u_max': u_max,
    'x_min': x_min,
    'x_max': x_max
}

# Run using DynamicalSystem
x_ds = np.zeros((N_steps, 2))
u_ds = np.zeros(N_steps)
x_ds[0] = x0

for k in range(N_steps - 1):
    param_dict['xk'] = x_ds[k]
    param_dict['xref'] = x_ref[k]
    
    # Step MPC controller
    (u_opt, x_pred), u_out = mpc_system.step(params=param_dict)
    param_dict['mpc_output'] = (u_out, x_pred)
    
    u_ds[k] = u_out
    x_ds[k+1] = A @ x_ds[k] + B @ np.array([u_out])

u_ds[-1] = u_ds[-2]

print("\nMPC integrated with DynamicalSystem:")
print(f"Final state: {x_ds[-1]}")
print(f"Matches standalone MPC: {np.allclose(x_ds, x_mpc)}")

## Conclusion

### Key Takeaways

1. **MPC handles constraints**: Explicitly respects input and state limits
2. **LQR ignores constraints**: Can violate limits and become unstable
3. **Receding horizon**: MPC re-plans at each step for optimality
4. **Computational cost**: MPC solves QP online, LQR is precomputed

### When to Choose MPC over LQR

| Scenario | MPC | LQR |
|----------|-----|-----|
| **Constraints** | ✓ Handles explicitly | ✗ Ignores |
| **Optimality** | ✓ Constrained optimal | ✓ Unconstrained optimal |
| **Computation** | High (online QP) | Low (precomputed) |
| **Time-varying** | ✓ Adaptive | ✗ Fixed gain |
| **Nonlinear** | ✓ Via linearization | ✗ Linear only |

### Practical Applications

- **Robotics**: Joint limits, collision avoidance
- **Aerospace**: Thrust limits, angular velocity constraints
- **Process control**: Temperature/pressure limits
- **Autonomous vehicles**: Steering/acceleration bounds

### Next Steps

- See `mpc_turtlebot.ipynb` for path following with obstacle avoidance
- See `mpc_crazyflie.ipynb` for aggressive flight maneuvers
- Experiment with different prediction horizons
- Try nonlinear MPC for complex dynamics

### Reference

Mayne, D. Q., Rawlings, J. B., Rao, C. V., & Scokaert, P. O. M. (2000). *Constrained model predictive control: Stability and optimality*. Automatica, 36(6), 789-814.